
Please Support Us By Giving A star On Github


In [ ]:
!pip install facebook-sdk pytz google-auth-oauthlib google-auth-httplib2 google-api-python-client

In [ ]:
import requests
import datetime
import time
import logging
from google.colab import auth
from googleapiclient.discovery import build
from google.colab import drive
import os
import pytz

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Mount Google Drive
drive.mount('/content/drive')

# Facebook Authentication
def get_user_pages(access_token):
    try:
        response = requests.get(f"https://graph.facebook.com/me/accounts?access_token={access_token}")
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        logger.error(f"Error fetching user pages: {e}")
        return None

# Google Drive Functions
def authenticate_google_drive():
    try:
        auth.authenticate_user()
        drive_service = build('drive', 'v3')
        return drive_service
    except Exception as e:
        logger.error(f"Error authenticating with Google Drive: {e}")
        return None

def list_drive_images(drive_service, folder_id):
    try:
        query = f"'{folder_id}' in parents and mimeType contains 'image/'"
        results = drive_service.files().list(q=query, pageSize=1000, fields="files(id, name)").execute()
        files = results.get('files', [])
        logger.info(f"Found {len(files)} images in the folder.")
        return files
    except Exception as e:
        logger.error(f"Error listing Drive images: {e}")
        return []

def delete_drive_file(drive_service, file_id):
    try:
        drive_service.files().delete(fileId=file_id).execute()
        logger.info(f"File with ID {file_id} has been deleted.")
        return True
    except Exception as e:
        logger.error(f"An error occurred while deleting file {file_id}: {e}")
        return False

# Scheduling Functions
def calculate_schedule(num_posts_per_day, first_post_time, time_gap_minutes, total_days):
    schedule = []
    current_time = first_post_time
    end_time = first_post_time + datetime.timedelta(days=min(total_days, 30))

    while current_time < end_time:
        for _ in range(num_posts_per_day):
            if current_time >= end_time:
                break
            schedule.append(current_time)
            current_time += datetime.timedelta(minutes=time_gap_minutes)

        # Move to the next day
        current_time = current_time.replace(hour=first_post_time.hour, minute=first_post_time.minute)
        current_time += datetime.timedelta(days=1)

    return schedule

def schedule_post(page_id, page_access_token, post_type, content, scheduled_time, caption=None, file_id=None):
    current_time = datetime.datetime.now(pytz.utc)
    if scheduled_time < current_time + datetime.timedelta(minutes=20) or \
       scheduled_time > current_time + datetime.timedelta(days=30):
        logger.error(f"Error: Post for {scheduled_time} must be scheduled between 20 minutes and 30 days from now.")
        return None

    if post_type == 'photo':
        url = f"https://graph.facebook.com/{page_id}/photos"
        payload = {
            'url': content,
            'access_token': page_access_token,
            'published': 'false',
            'scheduled_publish_time': int(scheduled_time.timestamp()),
            'caption': caption
        }
    else:  # text post
        url = f"https://graph.facebook.com/{page_id}/feed"
        payload = {
            'message': content,
            'access_token': page_access_token,
            'published': 'false',
            'scheduled_publish_time': int(scheduled_time.timestamp())
        }

    try:
        response = requests.post(url, data=payload)
        response.raise_for_status()
        response_json = response.json()

        logger.info(f"Post scheduled successfully for {scheduled_time}. Response: {response_json}")

        if 'error' in response_json:
            logger.error(f"Error in response: {response_json['error']}")

        return response_json
    except requests.RequestException as e:
        logger.error(f"Error scheduling post for {scheduled_time}: {e}")
        if e.response is not None:
            logger.error(f"Response content: {e.response.content}")
        return None

def get_scheduled_posts(page_id, page_access_token):
    url = f"https://graph.facebook.com/v21.0/{page_id}/scheduled_posts"
    params = {
        'access_token': page_access_token,
        'fields': 'id,message,scheduled_publish_time,status'
    }
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        scheduled_posts = response.json().get('data', [])
        logger.info(f"Retrieved {len(scheduled_posts)} scheduled posts")
        return scheduled_posts
    except requests.RequestException as e:
        logger.error(f"Error fetching scheduled posts: {e}")
        return None

def display_scheduled_posts(scheduled_posts):
    if not scheduled_posts:
        print("No scheduled posts found.")
        return

    print("\nScheduled Posts:")
    for post in scheduled_posts:
        post_id = post.get('id', 'N/A')
        message = post.get('message', 'N/A')
        scheduled_time = post.get('scheduled_publish_time', 'N/A')
        status = post.get('status', 'N/A')

        if scheduled_time != 'N/A':
            scheduled_time = datetime.datetime.fromtimestamp(scheduled_time).strftime('%Y-%m-%d %H:%M:%S')

        print(f"ID: {post_id}")
        print(f"Message: {message[:50]}..." if len(message) > 50 else f"Message: {message}")
        print(f"Scheduled Time: {scheduled_time}")
        print(f"Status: {status}")
        print("-" * 50)

# Main Function
def main():
    logger.info("Starting Facebook Post Scheduler")

    # User Authentication
    user_access_token = input("Enter your Facebook access token: ")
    pages = get_user_pages(user_access_token)
    if not pages:
        logger.error("Failed to retrieve Facebook pages. Exiting.")
        return

    print("Your Pages:")
    for page in pages['data']:
        print(f"{page['name']} (ID: {page['id']})")

    page_id = input("Enter the Page ID you want to post to: ")
    page_access_token = next((page['access_token'] for page in pages['data'] if page['id'] == page_id), None)
    if not page_access_token:
        logger.error("Invalid Page ID. Exiting.")
        return

    # Post Type Selection
    post_type = input("Enter the type of post (photo/text): ").lower()
    if post_type not in ['photo', 'text']:
        logger.error("Invalid post type. Please choose 'photo' or 'text'. Exiting.")
        return

    # Google Drive Folder Selection (for photo posts)
    drive_service = None
    files = []
    if post_type == 'photo':
        drive_service = authenticate_google_drive()
        if not drive_service:
            logger.error("Failed to authenticate with Google Drive. Exiting.")
            return

        while True:
            folder_id = input("Enter your Google Drive folder ID for images (or 'q' to quit): ")
            if folder_id.lower() == 'q':
                logger.info("User chose to quit. Exiting.")
                return

            files = list_drive_images(drive_service, folder_id)
            if files:
                break
            else:
                print("No images found or error occurred. Please check the folder ID and try again.")

        if not files:
            logger.error("Unable to proceed without images. Exiting.")
            return

        print(f"Found {len(files)} images in the folder.")

    # Timezone Selection
    user_timezone = input("Enter your timezone (e.g., 'US/Pacific', 'Europe/London'): ")
    try:
        tz = pytz.timezone(user_timezone)
    except pytz.exceptions.UnknownTimeZoneError:
        logger.error(f"Unknown timezone: {user_timezone}. Using UTC.")
        tz = pytz.UTC

    # Scheduling Inputs
    num_posts_per_day = int(input("How many posts do you want to schedule per day? "))
    first_post_time_str = input("Enter the date and time for the first post (YYYY-MM-DD HH:MM): ")
    first_post_time = tz.localize(datetime.datetime.strptime(first_post_time_str, "%Y-%m-%d %H:%M"))
    time_gap_minutes = int(input("Enter the time gap between each post (in minutes): "))
    total_days = int(input("Enter the total number of days to schedule posts for (max 30): "))

    schedule = calculate_schedule(num_posts_per_day, first_post_time, time_gap_minutes, total_days)

    if post_type == 'photo':
        if len(schedule) > len(files):
            logger.warning(f"More scheduling slots ({len(schedule)}) than images ({len(files)}). Will use all available images.")
        elif len(schedule) < len(files):
            logger.warning(f"More images ({len(files)}) than scheduling slots ({len(schedule)}). Some images will not be scheduled.")

    caption = input("Enter a caption for the posts: ")

    # Scheduling Process
    scheduled_posts = 0
    failed_posts = 0
    remaining_images = max(0, len(files) - len(schedule)) if post_type == 'photo' else 0

    for i, post_time in enumerate(schedule):
        if post_type == 'photo' and i >= len(files):
            break

        if post_type == 'text':
            content = caption
        else:  # photo
            file = files[i]
            content = f"https://drive.google.com/uc?id={file['id']}"

        logger.info(f"Attempting to schedule {post_type} post for {post_time}")
        response = schedule_post(page_id, page_access_token, post_type, content, post_time, caption if post_type == 'photo' else None, file['id'] if post_type == 'photo' else None)

        if response:
            scheduled_posts += 1
            if post_type == 'photo' and delete_drive_file(drive_service, file['id']):
                logger.info(f"Image {file['name']} deleted after successful scheduling.")
            else:
                logger.warning(f"Failed to delete image {file['name']} after scheduling.")
        else:
            failed_posts += 1

        # Respect API rate limits
        time.sleep(1)

    # Summary
    logger.info("Scheduling process completed")
    print("\nScheduling Summary:")
    print(f"Total posts attempted: {len(schedule)}")
    print(f"Posts scheduled successfully: {scheduled_posts}")
    print(f"Posts failed to schedule: {failed_posts}")
    if post_type == 'photo':
        print(f"Images remaining in folder: {remaining_images}")

    # Retrieve and display scheduled posts
    print("\nRetrieving list of scheduled posts...")
    scheduled_posts_list = get_scheduled_posts(page_id, page_access_token)
    if scheduled_posts_list:
        display_scheduled_posts(scheduled_posts_list)
    else:
        print("Unable to retrieve scheduled posts.")

if __name__ == "__main__":
    main()